In [1]:
# TODO: fix last dense layer. Is it the number of classes? If so then nothing to fix
# TODO: discuss how reading in dataset could be made multi-threaded (each dataset is read in on own thread then all joined together at the end)

import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import imageio as iio
import os
import xml.etree.ElementTree as ET
import cv2
import numpy as np
from PIL import Image
from numba import cuda  # https://stackoverflow.com/a/52354865/6476994
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import csv
import re
from datetime import datetime
import collections

In [2]:
# allows all images to be displayed at once (else only displays the last call to plt.imshow())
# https://stackoverflow.com/a/41210974
def displayImage(image, caption = None, colour = None) -> None:
    plt.figure()
    if(colour != None):
        plt.imshow(image, cmap=colour)
    else:
        plt.imshow(image)
        
    if(caption != None):
        # display caption below picture (https://stackoverflow.com/a/51486361)
        plt.figtext(0.5, 0.01, caption, wrap=True, horizontalalignment='center', fontsize=12)

In [3]:
# free up GPU if it didn't after the last run
cuda.select_device(0)
cuda.close()

# Read in dataset

In [4]:
%run readInDataset.py 20160724_July

args: ['readInDataset.py', '20160724_July']
dataset_names: ['20160724_July']
reading in images and labels for dataset: 20160724_July
all_folders_for_curr_dataset: ['BB01', 'BB02', 'BB03', 'BB04', 'BB05', 'BB06', 'BB07', 'BB08', 'BB09', 'BB10', 'BB11', 'BB12', 'BB13', 'BB14', 'BB15', 'BB16', 'BB17', 'BB18', 'BB19', 'BB20', 'BB21', 'BB22', 'BB23', 'BB24', 'BB25', 'BB26', 'BB27', 'BB28', 'BB29', 'BB30', 'BB31', 'BB32', 'BB33', 'BB34', 'BB35', 'BB36']
reading in images for subset: BB01
reading in labels for subset: BB01
done current subset
reading in images for subset: BB02
reading in labels for subset: BB02
done current subset
reading in images for subset: BB03
reading in labels for subset: BB03
done current subset
reading in images for subset: BB04
reading in labels for subset: BB04
done current subset
reading in images for subset: BB05
reading in labels for subset: BB05
done current subset
reading in images for subset: BB06
reading in labels for subset: BB06
done current subset
reading 

# ZFNet

Source: https://towardsdatascience.com/zfnet-an-explanation-of-paper-with-code-f1bd6752121d

## Train the model

In [5]:
# training_images = tf.map_fn(lambda i: tf.stack([i]*3, axis=-1), training_images).numpy()
# test_images = tf.map_fn(lambda i: tf.stack([i]*3, axis=-1), test_images).numpy()

training_images = tf.image.resize(training_images, [224, 224]).numpy()
test_images = tf.image.resize(test_images, [224, 224]).numpy()

training_images = training_images.reshape(training_images.shape)
training_images = training_images / 255.0
test_images = test_images.reshape(test_images.shape)
test_images = test_images / 255.0

training_labels = tf.keras.utils.to_categorical(training_labels, num_classes=len(training_classes))
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=len(test_classes))

num_len_train = int(0.8 * len(training_images))

ttraining_images = training_images[:num_len_train]
ttraining_labels = training_labels[:num_len_train]

valid_images = training_images[num_len_train:]
valid_labels = training_labels[num_len_train:]

training_images = ttraining_images
training_labels = ttraining_labels

model = tf.keras.models.Sequential([
                                    
		tf.keras.layers.Conv2D(96, (7, 7), strides=(2, 2), activation='relu',
			input_shape=(224, 224, 3)),
		tf.keras.layers.MaxPooling2D(3, strides=2),
    tf.keras.layers.Lambda(lambda x: tf.image.per_image_standardization(x)),

		tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), activation='relu'),
		tf.keras.layers.MaxPooling2D(3, strides=2),
    tf.keras.layers.Lambda(lambda x: tf.image.per_image_standardization(x)),

		tf.keras.layers.Conv2D(384, (3, 3), activation='relu'),

		tf.keras.layers.Conv2D(384, (3, 3), activation='relu'),

		tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),

		tf.keras.layers.MaxPooling2D(3, strides=2),

    tf.keras.layers.Flatten(),

		tf.keras.layers.Dense(4096),

		tf.keras.layers.Dense(4096),

		tf.keras.layers.Dense(len(classes), activation='softmax')#FIXME is this the number of classes? (check paper)
	])


model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9), \
              loss='categorical_crossentropy', \
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(5)])

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', \
                                            		factor=0.1, patience=1, \
																								min_lr=0.00001)
model.fit(training_images, training_labels, batch_size=32, \
          validation_data=(valid_images, valid_labels), \
					epochs=90, callbacks=[reduce_lr])

2022-05-29 10:59:41.172482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-29 10:59:41.175563: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-29 10:59:41.175736: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-29 10:59:41.176503: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/90


2022-05-29 10:59:50.752708: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-05-29 10:59:51.085298: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


140/140 [==============================] - 10s 59ms/step - loss: 1.2111 - accuracy: 0.5874 - top_k_categorical_accuracy: 0.9805 - val_loss: 1.0496 - val_accuracy: 0.6113 - val_top_k_categorical_accuracy: 0.9830 - lr: 0.0100
Epoch 2/90
140/140 [==============================] - 7s 50ms/step - loss: 1.0229 - accuracy: 0.6192 - top_k_categorical_accuracy: 0.9875 - val_loss: 0.9327 - val_accuracy: 0.6425 - val_top_k_categorical_accuracy: 0.9893 - lr: 0.0100
Epoch 3/90
140/140 [==============================] - 7s 50ms/step - loss: 0.9078 - accuracy: 0.6601 - top_k_categorical_accuracy: 0.9897 - val_loss: 0.8242 - val_accuracy: 0.7006 - val_top_k_categorical_accuracy: 0.9929 - lr: 0.0100
Epoch 4/90
140/140 [==============================] - 7s 50ms/step - loss: 0.8278 - accuracy: 0.6970 - top_k_categorical_accuracy: 0.9888 - val_loss: 0.7865 - val_accuracy: 0.7310 - val_top_k_categorical_accuracy: 0.9866 - lr: 0.0100
Epoch 5/90
140/140 [==============================] - 7s 51ms/step - loss:

## Evaluate the trained model

In [6]:
print('test_images shape: {}'.format(test_images.shape))
print('test_labels shape: {}'.format(test_labels.shape))

results = model.evaluate(test_images,test_labels)

test_images shape: (1398, 224, 224, 3)
test_labels shape: (1398, 8)
44/44 [==============================] - 1s 20ms/step - loss: 0.4171 - accuracy: 0.8784 - top_k_categorical_accuracy: 0.9986


2022-05-29 11:10:41.178618: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 698.12MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-05-29 11:10:41.178646: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 698.12MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


In [8]:
predictions = (model.predict(test_images) > 0.5).astype("int32")
# print("Predictions (shape: {}):\n{}".format(predictions.shape, predictions))

44/44 [==============================] - 1s 16ms/step


In [9]:
print("test_classes: {}".format(test_classes))
# classification_report uses alphabetic ordering of the classes, so to match the encoded labels to the target_names, provide a sortest list of classes
# https://stackoverflow.com/a/48495303
sorted_test_classes = sorted(test_classes)
print(classification_report(test_labels, predictions, target_names=sorted_test_classes))

test_classes: ['Human Presense/Deployment', 'Kangaroo', 'Empty photo', 'Fox', 'Emu', 'Cat', 'Rabbit', 'Other']
                           precision    recall  f1-score   support

                      Cat       1.00      0.50      0.67        16
              Empty photo       0.79      0.69      0.74       227
                      Emu       0.86      0.95      0.90       119
                      Fox       0.70      0.42      0.53        33
Human Presense/Deployment       0.90      0.74      0.81       110
                 Kangaroo       0.92      0.94      0.93       870
                    Other       0.67      0.38      0.48        16
                   Rabbit       0.00      0.00      0.00         7

                micro avg       0.89      0.86      0.87      1398
                macro avg       0.73      0.58      0.63      1398
             weighted avg       0.88      0.86      0.86      1398
              samples avg       0.86      0.86      0.86      1398



/home/luke/miniconda3/envs/cv-project-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Save the model
* use the current date/time so we can keep incrementation progress of the model as we re-run it

In [10]:
now = datetime.now()
dt_string = now.strftime('%d-%m-%Y_%H:%M:%S')
print("saving model as: 'ZFNet-{}.h5'.'".format(dt_string))

model.save('ZFNet-{}.h5'.format(dt_string))

saving model as: 'ZFNet-29-05-2022_11:11:12.h5'.'


## Free up the GPU's memory

In [11]:
cuda.select_device(0)
cuda.close()